In [18]:
import random
import math
import re

# 리스트 크기
listN = 8
# 염색체 해집단 수
N = 14

#교차율
Pc = 0.7
#변이율
Pm = 0.001

#마지막 세대수 
largest_generation = 500

def decoding(X, Y=0):
    #2번째 값부터 2진수를 10진수 화 (0부터 시작 -1, 처음값 제외 -1)
    for i, x in enumerate(X[1:]):
        Y += x* math.pow(2, listN -i -2)
    #처음 값이 0이면 음수로 return
    if X[0] == 0: return -Y
    return Y

In [2]:
def Readtxt(fname):
    #파일을 2차원 배열로 바꿔주기
    w = []
    with open(fname, 'r') as f :
        for line in f: #for each line in the file,
            w.append(line.split('\t'))
    return w

In [3]:
def ScoreFunction(X):
    """
    X에 속도 리스트를 넣어서 평가
    """
    AirportNum = Readtxt('data/Number-of-train-people.txt')
    AirportTime = Readtxt('data/Time-of-airport-train.txt')
    AtherTime = Readtxt('data/Time-of-ather-train.txt')
    sp = decoding(X)
    return 777 - (6 * sp)

In [5]:
def New_ScoreFunction(X):
    #여기다 바꾼 거를 만들면 됩니다.
    print(X)

In [13]:
#초기값
speedlist = [[random.choice([0,1]) for i in range(listN)]for j in range(N)]
speedlist

for sl in speedlist:
    print(ScoreFunction(sl), sl)

1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
951.0 [0, 0, 0, 1, 1, 1, 0, 1]
675.0 [1, 0, 0, 1, 0, 0, 0, 1]
261.0 [1, 1, 0, 1, 0, 1, 1, 0]
639.0 [1, 0, 0, 1, 0, 1, 1, 1]
591.0 [1, 0, 0, 1, 1, 1, 1, 1]
1365.0 [0, 1, 1, 0, 0, 0, 1, 0]
543.0 [1, 0, 1, 0, 0, 1, 1, 1]
93.0 [1, 1, 1, 1, 0, 0, 1, 0]
495.0 [1, 0, 1, 0, 1, 1, 1, 1]
849.0 [0, 0, 0, 0, 1, 1, 0, 0]
447.0 [1, 0, 1, 1, 0, 1, 1, 1]
183.0 [1, 1, 1, 0, 0, 0, 1, 1]
1053.0 [0, 0, 1, 0, 1, 1, 1, 0]


In [7]:
# 여기서 부터 세대 돌리게 만들어야 됨

def Group_assessment(speedlist):
    #speedlist, decoding값, 염색체 적합도, 적합도 비율
    speedlist_evaluation = [[speedlist[i], decoding(speedlist[i]), ScoreFunction(speedlist[i]), 0] for i in range(N)]
    # 적합도 비율 찾기
    # (각자의 값+이전값 총합)/총합
    Sum_of_fitness = 0
    for i in range(N): 
        Sum_of_fitness += speedlist_evaluation[i][2]
    for i in range(N):
        if i == 0:
            speedlist_evaluation[i][3] = speedlist_evaluation[i][2]/Sum_of_fitness
        else:
            speedlist_evaluation[i][3] = speedlist_evaluation[i-1][3]  + speedlist_evaluation[i][2]/Sum_of_fitness
    return speedlist_evaluation

In [8]:
# 0 ~ 100 룰랫 돌려서 이전 값과 자신 사이에 있으면 선택
def roulettes(Group_assessment_value):
    roulette = random.random()
    for i in range(N):
        if i == 0:
            if 0 < roulette < Group_assessment_value[i][3]:
                break
        if Group_assessment_value[i-1][3] < roulette < Group_assessment_value[i][3]:
            break
    return i

In [9]:
#부모 변경
def Change_parent(speedlist, Selected_list_in_roulette):
    W = []
    # 교차함수 및 변이함수 (자식만들기)
    for i in range(int(N/2)):
        selection = random.random()
        if selection < Pc:
            rand = random.choice([i+1 for i in range(listN-1)]) # 0~ 이구간, 이구간 ~ 끝 교차 , 이구간 = 1~리스트 크기 -1 
            
            # W(교차된 자식) = Selected_list_in_roulette(선택된 리스트에서) rand구간으로 나눈 값
            W.append(speedlist[Selected_list_in_roulette[i]][0:rand] + speedlist[Selected_list_in_roulette[i+1]][rand:])
            W.append(speedlist[Selected_list_in_roulette[i+1]][0:rand] + speedlist[Selected_list_in_roulette[i]][rand:])
            #print(i, W)
        else:
            W.append(speedlist[Selected_list_in_roulette[i]])
            W.append(speedlist[Selected_list_in_roulette[i+1]])
            #print(i, W)
    return W

In [35]:
#세대 반복
def Evolution(speedlist, num_generation):
    while 1: 
        if num_generation >= largest_generation:
            break

        print("-" * 15)    
        print(speedlist)
        print("-" * 15)    

        #적합도 찾고 룰렛 준비
        Group_assessment_value = Group_assessment(speedlist)

        #룰렛에 선택된 값 찾기 
        Selected_list_in_roulette = []

        #룰렛에 선택된 리스트들
        for i in range(N):
            Selected_list_in_roulette.append(roulettes(Group_assessment_value))

        New_list = Change_parent(speedlist, Selected_list_in_roulette)
        print("-" * 15)    
        print(num_generation,"번째를 합니다.")
        print("-" * 15)
        for i, nl in enumerate(New_list):
            print(i, ScoreFunction(nl), nl)
        
        speedlist = New_list
        num_generation += 1

    return New_list

   

In [36]:
test_list = []

test_list = Evolution(speedlist, 0)

---------------
[[0, 1, 0, 1, 0, 1, 1, 1], [0, 0, 0, 1, 1, 1, 0, 1], [1, 0, 0, 1, 0, 0, 0, 1], [1, 1, 0, 1, 0, 1, 1, 0], [1, 0, 0, 1, 0, 1, 1, 1], [1, 0, 0, 1, 1, 1, 1, 1], [0, 1, 1, 0, 0, 0, 1, 0], [1, 0, 1, 0, 0, 1, 1, 1], [1, 1, 1, 1, 0, 0, 1, 0], [1, 0, 1, 0, 1, 1, 1, 1], [0, 0, 0, 0, 1, 1, 0, 0], [1, 0, 1, 1, 0, 1, 1, 1], [1, 1, 1, 0, 0, 0, 1, 1], [0, 0, 1, 0, 1, 1, 1, 0]]
---------------
---------------
0 번째를 합니다.
---------------
0 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
1 675.0 [1, 0, 0, 1, 0, 0, 0, 1]
2 639.0 [1, 0, 0, 1, 0, 1, 1, 1]
3 675.0 [1, 0, 0, 1, 0, 0, 0, 1]
4 639.0 [1, 0, 0, 1, 0, 1, 1, 1]
5 951.0 [0, 0, 0, 1, 1, 1, 0, 1]
6 951.0 [0, 0, 0, 1, 1, 1, 0, 1]
7 1053.0 [0, 0, 1, 0, 1, 1, 1, 0]
8 951.0 [0, 0, 0, 1, 1, 1, 0, 1]
9 1053.0 [0, 0, 1, 0, 1, 1, 1, 0]
10 951.0 [0, 0, 0, 1, 1, 1, 0, 1]
11 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
12 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
13 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
---------------
[[0, 1, 0, 1, 0, 1, 1, 1], [1, 0, 0, 1, 0, 0, 0, 1], [1, 0, 0, 1, 0, 1, 

---------------
0 1293.0 [0, 1, 0, 1, 0, 1, 1, 0]
1 1347.0 [0, 1, 0, 1, 1, 1, 1, 1]
2 1341.0 [0, 1, 0, 1, 1, 1, 1, 0]
3 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
4 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
5 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
6 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
7 1341.0 [0, 1, 0, 1, 1, 1, 1, 0]
8 1347.0 [0, 1, 0, 1, 1, 1, 1, 1]
9 1293.0 [0, 1, 0, 1, 0, 1, 1, 0]
10 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
11 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
12 1293.0 [0, 1, 0, 1, 0, 1, 1, 0]
13 1347.0 [0, 1, 0, 1, 1, 1, 1, 1]
---------------
[[0, 1, 0, 1, 0, 1, 1, 0], [0, 1, 0, 1, 1, 1, 1, 1], [0, 1, 0, 1, 1, 1, 1, 0], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 1, 1, 1, 0], [0, 1, 0, 1, 1, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 0], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 0], [0, 1, 0, 1, 1, 1, 1, 1]]
---------------
---------------
14 번째를 합니다.
---------------
0 1341.0 [0, 1, 0, 1, 1, 1, 1, 0]
1 1347.0 [0, 1, 0, 1, 1, 1, 1

6 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
7 1347.0 [0, 1, 0, 1, 1, 1, 1, 1]
8 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
9 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
10 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
11 1347.0 [0, 1, 0, 1, 1, 1, 1, 1]
12 1347.0 [0, 1, 0, 1, 1, 1, 1, 1]
13 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
---------------
[[0, 1, 0, 1, 1, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 1, 1, 1, 1], [0, 1, 0, 1, 1, 1, 1, 1], [0, 1, 0, 1, 1, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 1, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 1, 1, 1, 1], [0, 1, 0, 1, 1, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1]]
---------------
---------------
24 번째를 합니다.
---------------
0 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
1 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
2 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
3 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
4 1347.0 [0, 1, 0, 1, 1, 1, 1, 1]
5 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
6 1347.0 [0, 1, 0, 1, 1, 1, 1, 1]
7 1347.0 [0, 1, 0, 1, 1, 1, 1, 1]
8 1347.0 [0

7 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
8 1347.0 [0, 1, 0, 1, 1, 1, 1, 1]
9 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
10 1347.0 [0, 1, 0, 1, 1, 1, 1, 1]
11 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
12 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
13 1347.0 [0, 1, 0, 1, 1, 1, 1, 1]
---------------
[[0, 1, 0, 1, 1, 1, 1, 1], [0, 1, 0, 1, 1, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 1, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 1, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 1, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 1, 1, 1, 1]]
---------------
---------------
35 번째를 합니다.
---------------
0 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
1 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
2 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
3 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
4 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
5 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
6 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
7 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
8 1347.0 [0, 1, 0, 1, 1, 1, 1, 1]
9 1299.0 [0

---------------
---------------
46 번째를 합니다.
---------------
0 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
1 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
2 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
3 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
4 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
5 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
6 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
7 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
8 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
9 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
10 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
11 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
12 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
13 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
---------------
[[0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1]]
---------------
---------------
47 번째를 합니다.
---------------
0 1299.0 [0, 1, 0, 

...

12 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
13 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
---------------
[[0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1]]
---------------
---------------
445 번째를 합니다.
---------------
0 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
1 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
2 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
3 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
4 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
5 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
6 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
7 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
8 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
9 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
10 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
11 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
12 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
13 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
--------

9 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
10 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
11 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
12 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
13 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
---------------
[[0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1]]
---------------
---------------
463 번째를 합니다.
---------------
0 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
1 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
2 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
3 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
4 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
5 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
6 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
7 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
8 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
9 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
10 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
11 1299.0

13 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
---------------
[[0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1]]
---------------
---------------
477 번째를 합니다.
---------------
0 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
1 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
2 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
3 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
4 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
5 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
6 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
7 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
8 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
9 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
10 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
11 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
12 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
13 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
---------------
[[0, 1, 0, 1, 0, 1, 1, 1], 

4 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
5 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
6 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
7 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
8 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
9 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
10 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
11 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
12 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
13 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
---------------
[[0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1]]
---------------
---------------
488 번째를 합니다.
---------------
0 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
1 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
2 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
3 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
4 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
5 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
6 1299.0 [

6 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
7 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
8 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
9 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
10 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
11 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
12 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
13 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
---------------
[[0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1]]
---------------
---------------
499 번째를 합니다.
---------------
0 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
1 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
2 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
3 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
4 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
5 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
6 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
7 1299.0 [0, 1, 0, 1, 0, 1, 1, 1]
8 1299.0 [

In [37]:
print(test_list)

[[0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 1]]


In [2]:
a = Readtxt( 'data/Number-of-train-people.txt')

In [3]:
a[0][3]

'17683'

In [4]:
a

[['서울',
  '일반',
  '9516',
  '17683',
  '27199',
  '8170',
  '18430',
  '26600',
  '53799',
  '\n'],
 ['공덕',
  '일반',
  '1674',
  '12842',
  '14516',
  '2140',
  '13366',
  '15506',
  '30022',
  '\n'],
 ['홍대',
  '일반',
  '9088',
  '19332',
  '28420',
  '9007',
  '21731',
  '30738',
  '59158',
  '\n'],
 ['미디어시티',
  '일반',
  '11639',
  '11423',
  '23062',
  '10200',
  '12101',
  '22301',
  '45363',
  '\n'],
 ['김포공항',
  '일반',
  '10393',
  '30080',
  '40473',
  '6548',
  '34747',
  '41295',
  '81768',
  '\n'],
 ['계양',
  '일반',
  '10865',
  '18624',
  '29489',
  '10130',
  '18776',
  '28906',
  '58395',
  '\n'],
 ['검암',
  '일반',
  '9144',
  '14121',
  '23265',
  '9039',
  '12846',
  '21885',
  '45150',
  '\n'],
 ['청라', '일반', '5323', '0', '5323', '5121', '0', '5121', '10444', '\n'],
 ['영종', '일반', '1869', '0', '1869', '1831', '0', '1831', '3700', '\n'],
 ['운서', '일반', '5724', '0', '5724', '5795', '0', '5795', '11519', '\n'],
 ['화물청사', '일반', '2619', '0', '2619', '2899', '0', '2899', '5518', '\n'],
 [